In [ ]:
import random
from faker import Faker
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

fake = Faker()

# Define constants
company_names = [fake.company() for _ in range(10)]
countries = [fake.country() for _ in range(10)]
lines_of_business = ["Retail", "Manufacturing", "Technology", "Healthcare", "Finance"]
company_sizes = ["Small", "Medium", "Large"]
revenue_types = ["Product Sales", "Service Revenue", "Subscription"]
revenue_streams = ["Online", "Offline"]
regions = ["North America", "Europe", "Asia", "Australia", "Africa"]
payment_methods = ["Credit Card", "Bank Transfer", "Cash", "PayPal"]
quality_categories = ["High", "Medium", "Low"]
transaction_statuses = ["Completed", "Pending", "Failed"]
loan_statuses = ["Approved", "Pending", "Rejected"]
loan_flags = ["Yes", "No"]
product_prices = {}

# Define product names relevant to categories
product_categories = {
    "Books": ["Novel", "Textbook", "Magazine", "Comic", "Manual"],
    "Clothing": ["Shirt", "Pants", "Dress", "Jacket", "Shoes"],
    "Electronics": ["Laptop", "Smartphone", "Tablet", "Camera", "Headphones"],
    "Furniture": ["Chair", "Table", "Sofa", "Bed", "Cabinet"],
    "Household Items": ["Vacuum", "Blender", "Microwave", "Toaster", "Iron"]
}

# Generate company records
company_records = []

for _ in range(1000):
    company_name = random.choice(company_names)
    country = random.choice(countries)
    line_of_business = random.choice(lines_of_business)
    company_size = random.choice(company_sizes)
    date = fake.date()
    revenue_type = random.choice(revenue_types)
    revenue_stream = random.choice(revenue_streams)
    currency = "USD"
    fiscal_year = random.randint(2018, 2023)
    fiscal_quarter = random.randint(1, 4)
    region = random.choice(regions)
    payment_method = random.choice(payment_methods)
    product_category = random.choice(list(product_categories.keys()))
    product_name = random.choice(product_categories[product_category])

    # Ensure consistent product price
    if product_name not in product_prices:
        product_prices[product_name] = round(random.uniform(10, 1000), 2)
    product_price = product_prices[product_name]

    quality_category = random.choice(quality_categories)
    import_percentage = round(random.uniform(0, 100), 2)
    export_percentage = round(random.uniform(0, 100), 2)
    transaction_amount = round(random.uniform(100, 10000), 2)
    transaction_date = fake.date()
    transaction_status = random.choice(transaction_statuses)
    loan_id = fake.uuid4()
    loan_status = random.choice(loan_statuses)
    principal_amount = round(random.uniform(1000, 100000), 2)
    terms = random.randint(1, 60)
    effective_date = fake.date()
    due_date = fake.date()
    paid_off_time = fake.date()
    past_due_days = random.randint(0, 30)
    loan_flag = random.choice(loan_flags)

    company_records.append([
        company_name, country, line_of_business, company_size, date, revenue_type, revenue_stream, currency, fiscal_year, fiscal_quarter, region, payment_method, transaction_amount, transaction_date, transaction_status, loan_id, loan_status, principal_amount, terms, effective_date, due_date, paid_off_time, past_due_days, loan_flag, product_name, product_category, product_price, quality_category, import_percentage, export_percentage
    ])

# Create DataFrame
columns = ['company_name', 'country', 'Line_of_business', 'company_size', 'date', 'revenue_type', 'revenue_stream', 'currency', 'fiscal_year', 'fiscal_quarter', 'region', 'payment_method', 'transaction_amount', 'transaction_date', 'transaction_status', 'Loan_ID', 'loan_status', 'Principal_amount', 'terms', 'effective_date', 'due_date', 'paid_off_time', 'past_due_days', 'Loan_flag', 'product_name', 'product_category', 'product_price', 'quality_category', 'Import percentage', 'export percentage']
df = pd.DataFrame(company_records, columns=columns)
df.to_csv('company_data.csv', index=False)
# Aggregate transaction_amount to calculate net_revenue and tax_paid
net_revenue_df = df.groupby(['company_name', 'fiscal_year'])['transaction_amount'].sum().reset_index()
net_revenue_df['net_revenue'] = net_revenue_df['transaction_amount']
net_revenue_df['tax_paid'] = net_revenue_df['net_revenue'] * 0.10
net_revenue_df = net_revenue_df.drop(columns=['transaction_amount'])

# Merge net_revenue and tax_paid back to the main DataFrame
df = df.merge(net_revenue_df, on=['company_name', 'fiscal_year'])

# Ensure 'loan_flag' column exists and has correct values
df['loan_flag'] = df['Loan_flag'].map({'Yes': 1, 'No': 0})

# Feature engineering
df['company_size'] = df['company_size'].map({'Small': 0, 'Medium': 1, 'Large': 2})
df['loan_status'] = df['loan_status'].map({'Approved': 1, 'Pending': 0, 'Rejected': -1})
df['performance'] = (df['past_due_days'] == 0).astype(int) * 100  # Scale performance to 1-100

# Select features and target
features = ['company_size', 'net_revenue', 'loan_status', 'loan_flag']
target = 'performance'

# Split the data
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")

# Make predictions and suggest schemes
df['predicted_performance'] = model.predict(X)
df['predicted_performance'] = df['predicted_performance'].clip(0, 100)  # Ensure predictions are within 0-100

# Round performance to the nearest multiple of 10
df['predicted_performance'] = (df['predicted_performance'] / 10).round() * 10

# Define retention schemes based on performance value
def suggest_scheme(performance):
    if performance <= 0.5:
        return "No scheme"
    elif performance <= 1:
        return "Basic retention scheme"
    elif performance <= 1.5:
        return "Intermediate retention scheme"
    elif performance <= 2.0:
        return "Advanced retention scheme"
    else:
        return "Premium retention scheme"
df = df.groupby('company_name').agg({
    'predicted_performance': 'mean',
}).reset_index()
df['suggested_scheme'] = df['predicted_performance'].apply(suggest_scheme)

# Group by company name and get the average performance and scheme


# Select only the required columns
output_df = df[['company_name', 'predicted_performance', 'suggested_scheme']]

# Save the results
output_df.to_csv('company_performance_predictions.csv', index=False)

print("Model trained and predictions saved to 'company_performance_predictions.csv'")